## Objectifs du notebook
- Visualiser les relations entre les mots utilisés dans chaque moment de la dissolution

In [9]:
# On charge les données du corpus

import pandas as pd
import pickle

pd.set_option('max_colwidth',120)

data = pd.read_pickle('corpus_WE.pkl')
data

,messages
GI_pré-dissolution,bon année à tout que 2021 être un excellent année pour tout le défenseur de le identité avoir le abordage \n 📢 l...
GI_dissolution,darmanin être scandaliser que de jeune français avoir réclamer le application de le loi le entrée de migrant cl...
TE_pré-dissolution,merci à vous tout qui me rejoindre sur telegram le monopole de gafa sur internet n avoir que trop durer le mand...
TE_dissolution,vous souhaiter nous aider alors ce être le moment de agir darmanin avoir annoncer vouloir dissoudre génération id...
TE_post-dissolution,🗣 photo prendre ce matin alors que le force de l ordre nous bloquer l accès au pont où nous souhaiter faire notre ...


Pour chaque moment de la dissolution, on associe dans un dictionnaire une liste contenant elle-même des listes lemmatisées et nettoyées pour chaque message.

In [10]:
sent = []
sent_temp = []
data_dict = {}
a = 0

for row in data['messages']:
    
    for w in row.split(' '):
        if w == '\n':
            sent.append(sent_temp)
            sent_temp = []
        elif w != '\xa0':
            sent_temp.append(w)

    data_dict[data.index[a]] = sent
    sent = []
    a += 1

print(data_dict)

{'GI_pré-dissolution': [['bon', 'année', 'à', 'tout', '', 'que', '2021', 'être', 'un', 'excellent', 'année', 'pour', 'tout', 'le', 'défenseur', 'de', 'le', 'identité', '', '', 'avoir', 'le', 'abordage', ''], ['📢', 'le', 'vrai', 'visage', 'de', 'celui', 'qui', 'demander', 'le', 'dissolution', 'de', 'génération', 'identitaire', 'et', 'le', 'fermeture', 'de', 'le', 'traboule', '', 'votre', 'maison', 'de', 'l', '', 'identité', '⬇', '️'], ['en', '2012', '', 'génération', 'identitaire', 'se', 'être', 'faire', 'connaître', 'grâce', 'à', 'ce', 'vidéo', '⬆', '️', 'intituler', '', '', 'le', 'déclaration', 'de', 'guerre', '', '', '', '', 'connaître', 'un', 'très', 'vif', 'succès', 'en', 'france', '', 'lui', 'avoir', 'ensuite', 'être', 'spontanément', 'traduire', 'dans', 'différent', 'pays', 'européen', '', '', 'ce', 'élan', 'avoir', 'participer', 'ainsi', 'au', 'développement', 'de', 'mouvement', 'identitaire', 'sur', 'tout', 'notre', 'continent', '', ''], ['gavées', 'd', '', 'argent', 'public', 

In [11]:
# Construction du modèle word2vec

import gensim

w2v_model = gensim.models.Word2Vec(data_dict['TE_post-dissolution'], 
                                   vector_size=100,
                                   min_count=2, 
                                   sg = 1,
                                   workers=4)

In [12]:
# Construction de la liste de mots à visualiser

import spacy
from nltk.corpus import stopwords

stopwords_list = stopwords_list = stopwords.words('french')

nlp = spacy.load('fr_core_news_md')
vocab = []
doc = nlp(' '.join(w2v_model.wv.index_to_key))

# On va visualiser les noms et les adjectifs dans le corpus
for tok in doc:
    if tok.pos_ == "NOUN" or tok.pos_ == "ADJ" and tok.text not in vocab and tok.text not in stopwords_list:
        vocab.append(tok.text)

vocab

['d',
 'l',
 'identitaire',
 'conseil',
 'lui',
 'dissolution',
 'génération',
 'question',
 'état',
 'jeune',
 'n',
 'politique',
 'france',
 'autre',
 'hui',
 'martin',
 'tribune',
 'mars',
 '➡',
 'grand',
 'français',
 'entretien',
 '⤵️',
 'droit',
 'immigration',
 '►',
 'europe',
 'merci',
 'propre',
 'c',
 'matin',
 'vidéo',
 'charlotte',
 'femme',
 'action',
 'suite',
 'présent',
 'place',
 'répression',
 'mois',
 'propos',
 'face',
 'vérité',
 'valeur',
 'lecture',
 'etat',
 'audience',
 'vendredi',
 'défenseur',
 'scandaleux',
 'pays',
 'association',
 'haine',
 'pierre',
 'hamida',
 'remigration',
 'dernier',
 'suspension',
 'fond',
 'mesure',
 'décision',
 'insécurité',
 'voltaire',
 'abdelaziz',
 'événement',
 'boulevard',
 'intéressant',
 'avocat',
 'correct',
 'anciennement',
 'maire',
 'fois',
 'moment',
 'octobre',
 'scandale',
 'député',
 'chaîne',
 'islamiste',
 'photo',
 '🏻\u200d',
 'force',
 'élément',
 'liberté',
 'ministère',
 'émission',
 'avril',
 'journaliste',


In [13]:
# Visualisation

%matplotlib qt
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
from sklearn.decomposition import PCA

def display_words_vectors_pca(model, words):
    word_vectors = np.array([w2v_model.wv[w] for w in words])
    twodim = PCA().fit_transform(word_vectors)[:,:2]
    
    plt.figure(figsize=(30,30))
    plt.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
    plt.title('TE : dissolution', fontsize=25)
    for word, (x,y) in zip(words, twodim):
        plt.text(x, y+0.0001, word)

In [14]:
display_words_vectors_pca(w2v_model, vocab)

C:\Users\verny\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 10549 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\verny\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:201: RuntimeWarning: Glyph 10549 missing from current font.
  font.set_text(s, 0, flags=flags)
C:\Users\verny\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:238: RuntimeWarning: Glyph 127995 missing from current font.
  font.set_text(s, 0.0, flags=flags)
C:\Users\verny\anaconda3\lib\site-packages\matplotlib\backends\backend_agg.py:201: RuntimeWarning: Glyph 127995 missing from current font.
  font.set_text(s, 0, flags=flags)


### Note : dans la mesure où les word embedding manquaient de précision et de pertinence pour mon analyse, j'ai finalement décidé de ne pas les utiliser.